In [1]:
import uuid
import pandas
import numpy
from random import shuffle

import sklearn
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import sklearn.manifold

import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import os

In [103]:
def load_data():
    
    filename = "/Users/simondi/PHD/data/data/target_infect_x/screening_data/infectx-bartonella-d-p-g-1-dz14-2a_cells_data.tsv"

    data = pandas.read_csv(filename, sep="\t", header=0)

    feature_cols = [(i, x) for i, x in enumerate(data.columns.values) if
                    x.startswith("cells")]
    for i, c in feature_cols:
        data.loc[:, c] = data.loc[:, c].astype('float64')

    X = data.dropna(subset=["cells.children_invasomes_count"], how='all')
    X["cells.children_invasomes_count"] =  X["cells.children_invasomes_count"].fillna(0)
    X = X.dropna()
    X["infection"] = X["cells.children_invasomes_count"]
    del X["cells.children_invasomes_count"]
    
    return X

In [3]:
def plot(data, gene_list, cnt):

    shuffle(gene_list)
    used_genes = gene_list[1:cnt]
    X = data[ data["gene"].isin(used_genes)]
     
    uniq = list(set(X["gene"]))

    hot = plt.get_cmap('hot')
    cNorm = colors.Normalize(vmin=0, vmax=len(uniq))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=hot)
    alpha = 0.5

    plt.figure()
    for i in range(len(uniq)):
        indx = X["gene"] == uniq[i]
        plt.scatter(X.loc[indx, "cells.location_center_x"], X.loc[indx, "cells.location_center_x"],
                    color=scalarMap.to_rgba(i),
                    label=uniq[i], marker=".", alpha=alpha)
    if len(uniq) < 20:     
        plt.legend()    
    
    uid = str(uuid.uuid1())
    plt.show()

In [112]:
X = load_data()

In [113]:
feature_cols = [x for x in X.columns if x.startswith("cells")]
for c in feature_cols:
    X.loc[:, c] = (X.loc[:, c] - numpy.mean(X.loc[:, c])) / numpy.std(X.loc[:, c])

In [126]:
for c in feature_cols:
    print(X[c].mean(), " ", X[c].std())

-7.96226897972e-17   1.00000137327
-5.79372160465e-16   1.00000137327
2.54605259846e-15   1.00000137327
-4.06856332571e-16   1.00000137327
-2.42771142421e-16   1.00000137327
-3.37069386808e-16   1.00000137327
-1.30948100132e-17   1.00000137327
4.42296235677e-16   1.00000137327
1.62055592175e-16   1.00000137327
-3.8406238608e-16   1.00000137327
1.46131054216e-16   1.00000137327
-1.01448674177e-15   1.00000137327
4.97407626851e-16   1.00000137327
-3.93585884272e-16   1.00000137327
1.54311895285e-15   1.00000137327
8.70853654291e-16   1.00000137327
-4.85210523634e-16   1.00000137327
3.47763806908e-16   1.00000137327
4.6805651767e-16   1.00000137327
3.71572552387e-17   1.00000137327
-1.14594224139e-15   1.00000137327
-2.99229093789e-16   1.00000137327
3.49403097581e-16   1.00000137327
-2.18572089639e-18   1.00000137327
3.03581020217e-16   1.00000137327
1.68144386101e-16   1.00000137327
1.01292551256e-15   1.00000137327
4.25434960191e-16   1.00000137327
3.32112484061e-16   1.00000137327
-5.

In [114]:
del grp

NameError: name 'grp' is not defined

In [116]:
gene_list = list(set(X["gene"]))
len(gene_list)

304

In [117]:
well_list = list(set(X["well"]))
len(well_list)

384

In [118]:
cnt  = 10

In [119]:
shuffle(well_list)
used_genes = well_list[0:cnt]

# take mid images
X_5well = X.loc[X.image_idx % 5 == 0,]

X_5well_wells = list(set(X_5well.well))
shuffle(X_5well_wells)

# take random 5 genes/weels
Xw = X_5well.loc[X_5well["well"].isin(X_5well_wells[0:5]), :]

In [ ]:
Xw.groupby("well")

In [122]:
fl = "/Users/simondi/PROJECTS/target_infect_x_project/src/tix_util/tix_analysis/plots/scatter_"

In [125]:
uid =str(uuid.uuid1())

'e06d32dc-536b-11e7-a1a8-a45e60c45f0f'

In [127]:
uniq = list(set(Xw.gene))

hot = plt.get_cmap('hot')
cNorm = colors.Normalize(vmin=0, vmax=len(uniq))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=hot)
alpha = 0.5

# plot features pairwisely
for c1 in feature_cols:
    for c2 in feature_cols:
        fig = plt.figure(0)
        ax = fig.add_subplot(111)
        for i in range(len(uniq)):
            indx = Xw["gene"] == uniq[i]
        
            plt.scatter(Xw.loc[indx, c1],
                        Xw.loc[indx, c2],
                        color=scalarMap.to_rgba(i),
                        label=uniq[i], marker=".", alpha=alpha)
            if len(uniq) < 20:     
                plt.legend()    
        plt.xlabel(c1)
        plt.ylabel(c2)
        plt.savefig(
          fl + "_plate_bartonella-d-p-g-1-dz14-2a_" + c1 + "_"+c2 +"_" +uid+ ".png",
          dpi=720)
        plt.close(fig)


SystemError: <built-in method write of _io.BufferedWriter object at 0x139db6728> returned a result with an error set

In [128]:
plt.close(0)

In [137]:
def plot_dimred(X2, X, method):
    uniq = list(set(X["gene"]))
    
    hot = plt.get_cmap('hot')
    cNorm = colors.Normalize(vmin=0, vmax=len(uniq))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=hot)
    alpha = 0.5
    
    fig = plt.figure()
    for i in range(len(uniq)):
        indx = X["gene"] == uniq[i]
        plt.scatter(X2[indx, 1], X2[indx, 0], color=scalarMap.to_rgba(i),
                    label=uniq[i], marker=".", alpha=alpha)
    if len(uniq) < 20:     
        plt.legend()
    plt.savefig(fl + "_" + method + "_2d" +uid+ ".png", dpi=720)
    plt.close(fig)

In [138]:
feature_cols_idx = [x for x in (Xw.columns.values) if x.startswith("cells")]
perp = [2,5,10,30,50]
for x in perp:
    ts2 = sklearn.manifold.TSNE(n_components=2, perplexity=x, n_iter=5000)
    X2 = ts2.fit_transform(Xw.loc[:, feature_cols_idx])
    plot_dimred(X2, Xw, "_plate_bartonella-d-p-g-1-dz14-2a_tsne_" + str(x))

In [142]:
for x in ["rbf", "poly", "rbf"]:
    ts2 = sklearn.decomposition.KernelPCA(n_components=2, kernel=x)
    X2 = ts2.fit_transform(Xw.loc[:, feature_cols_idx])
    plot_dimred(X2, Xw, "_plate_bartonella-d-p-g-1-dz14-2a_kpca_" + x)

In [141]:
ts2 = sklearn.decomposition.PCA(n_components=2, )
X2 = ts2.fit_transform(Xw.loc[:, feature_cols_idx])
plot_dimred(X2, Xw, "_plate_bartonella-d-p-g-1-dz14-2a_pca_")